In [11]:
import numpy as np
import pandas as pd
import networkx as nx
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import laplacian
from scipy.sparse.linalg import eigsh
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Gewichtungsfaktoren
risk_weights = {
    "both": 393,
    "one": 18,
    "none": 2,
}

# Daten bereinigen
def clean_preferences(preferences_df):
    preferences_df = preferences_df.fillna(0)  # Fehlende Werte mit 0 auffüllen
    return preferences_df.astype(int)  # Alles in Integer konvertieren

# Gewicht für eine Kante berechnen
def calculate_edge_weight(user1, user2, preferences_df):
    try:
        # Prüfe, ob Präferenzdaten existieren
        if user1 not in preferences_df.index or user2 not in preferences_df.index:
            raise ValueError(f"Präferenzdaten fehlen für Benutzer {user1} oder {user2}.")

        # Konvertiere Präferenzen in Binärdaten
        genres_user1 = np.array(preferences_df.loc[user1].values, dtype=int)
        genres_user2 = np.array(preferences_df.loc[user2].values, dtype=int)

        # Sichere Bit-Operationen
        common_genres = np.sum(genres_user1 & genres_user2)
        one_sided_genres = np.sum(genres_user1 ^ genres_user2)
        neither_genres = len(genres_user1) - (common_genres + one_sided_genres)

        # Begrenze extrem hohe Werte
        max_genres = 1e3  # Beispiel-Grenzwert
        common_genres = min(common_genres, max_genres)
        one_sided_genres = min(one_sided_genres, max_genres)
        neither_genres = max(0, len(genres_user1) - (common_genres + one_sided_genres))

        # Kanten-Gewicht berechnen
        weight = (
            risk_weights["both"] * common_genres +
            risk_weights["one"] * one_sided_genres +
            risk_weights["none"] * neither_genres
        )
        return weight
    except Exception as e:
        print(f"Fehler beim Berechnen des Gewichts für Kante ({user1}, {user2}): {e}")
        return 0



# Netzwerk und Präferenzen laden
friends_df = pd.read_csv('friends.csv', delimiter=',', header=None)
G = nx.Graph()
G.add_edges_from(friends_df.values)
print(f"Netzwerk erstellt: {G.number_of_nodes()} Knoten, {G.number_of_edges()} Kanten")

preferences_df = pd.read_json('preferences.json', orient='index')
preferences_df = clean_preferences(preferences_df)

# Kanten mit Gewichten versehen
print("Berechnung der Kanten-Gewichte...")
weighted_edges = []
for user1, user2 in G.edges():
    weight = calculate_edge_weight(user1, user2, preferences_df)
    weighted_edges.append((user1, user2, weight))

G.add_weighted_edges_from(weighted_edges)

# Laplace-Matrix berechnen
print("Berechnung der Laplace-Matrix...")
A_sparse = nx.to_scipy_sparse_array(G, format="csr")
L_sparse = laplacian(A_sparse, normed=True)

# Spektrale Analyse
print("Berechnung der Eigenwerte und Eigenvektoren...")
k = 10  # Clusteranzahl
eigvals, eigvecs = eigsh(L_sparse, k=k+1, which="SM")
embedding = eigvecs[:, 1:k+1]  # Erste Spalte (triviale Lösung) überspringen

# K-Means-Clustering
print("Clustering der Benutzer...")
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(embedding)

# Degree Centrality
print("Berechnung der Degree Centrality...")
degree_centrality = nx.degree_centrality(G)

# Auswahl der wichtigsten Knoten
print("Auswahl der Impfkandidaten...")
cluster_nodes = {i: [] for i in range(k)}
for node, cluster in zip(G.nodes(), clusters):
    cluster_nodes[cluster].append((node, degree_centrality[node]))

selected_nodes = []
for cluster, nodes in cluster_nodes.items():
    nodes = sorted(nodes, key=lambda x: x[1], reverse=True)  # Sortiere nach Degree Centrality
    selected_nodes.extend([node for node, _ in nodes[:int(997 / k)]])

# Ergebnisse speichern
with open("selected_vaccination_nodes.txt", "w") as f:
    for node in selected_nodes:
        f.write(f"{node}\n")
print(f"Ausgewählte Knoten gespeichert: {len(selected_nodes)} Knoten.")

# Visualisierung
pos = nx.spring_layout(G, seed=42)
colors = ['red' if node in selected_nodes else 'blue' for node in G.nodes()]
plt.figure(figsize=(12, 8))
nx.draw(G, pos, node_color=colors, node_size=10, edge_color='gray', alpha=0.7)
plt.title("Netzwerkvisualisierung mit hervorgehobenen Teilnehmern")
plt.show()


Netzwerk erstellt: 8311 Knoten, 55483 Kanten
Berechnung der Kanten-Gewichte...
Warnung: Extreme Werte für Kante (360, 4719).
Warnung: Extreme Werte für Kante (360, 4720).
Warnung: Extreme Werte für Kante (360, 4721).
Warnung: Extreme Werte für Kante (360, 4722).
Warnung: Extreme Werte für Kante (360, 14).
Warnung: Extreme Werte für Kante (360, 4723).
Warnung: Extreme Werte für Kante (360, 1020).
Warnung: Extreme Werte für Kante (360, 4724).
Warnung: Extreme Werte für Kante (360, 4725).
Warnung: Extreme Werte für Kante (360, 4726).
Warnung: Extreme Werte für Kante (360, 4727).
Warnung: Extreme Werte für Kante (360, 4728).
Warnung: Extreme Werte für Kante (360, 4729).
Warnung: Extreme Werte für Kante (360, 1286).
Warnung: Extreme Werte für Kante (360, 4730).
Warnung: Extreme Werte für Kante (360, 4731).
Warnung: Extreme Werte für Kante (360, 4732).
Warnung: Extreme Werte für Kante (360, 4733).
Warnung: Extreme Werte für Kante (360, 4734).
Warnung: Extreme Werte für Kante (360, 4735).
War

/var/folders/sj/tgql6qpd47j0ffyrwhssd1tw0000gp/T/ipykernel_36924/4180327057.py:42: RuntimeWarning: overflow encountered in scalar multiply
  risk_weights["none"] * neither_genres


Warnung: Extreme Werte für Kante (1337, 2930).
Warnung: Extreme Werte für Kante (1337, 1447).
Warnung: Extreme Werte für Kante (1337, 694).
Warnung: Extreme Werte für Kante (1337, 7263).
Warnung: Extreme Werte für Kante (1337, 3210).
Warnung: Extreme Werte für Kante (1337, 6512).
Warnung: Extreme Werte für Kante (1337, 5380).
Warnung: Extreme Werte für Kante (1337, 6957).
Warnung: Extreme Werte für Kante (1337, 1437).
Warnung: Extreme Werte für Kante (1337, 5338).
Warnung: Extreme Werte für Kante (1337, 4475).
Warnung: Extreme Werte für Kante (1337, 1499).
Warnung: Extreme Werte für Kante (1337, 7139).
Warnung: Extreme Werte für Kante (1337, 7784).
Warnung: Extreme Werte für Kante (1337, 5655).
Warnung: Extreme Werte für Kante (1337, 5700).
Warnung: Extreme Werte für Kante (1337, 1493).
Warnung: Extreme Werte für Kante (1337, 79).
Warnung: Extreme Werte für Kante (1337, 4847).
Warnung: Extreme Werte für Kante (1337, 5642).
Warnung: Extreme Werte für Kante (1337, 1429).
Warnung: Extreme

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/sparse/csgraph/_laplacian.py:470: RuntimeWarning: invalid value encountered in sqrt
  w = np.where(isolated_node_mask, 1, np.sqrt(w))


Berechnung der Eigenwerte und Eigenvektoren...
Clustering der Benutzer...
Berechnung der Degree Centrality...
Auswahl der Impfkandidaten...
Ausgewählte Knoten gespeichert: 990 Knoten.


KeyboardInterrupt: 